In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import glob
from re import T
from sympy import *
from scipy.spatial.transform import Rotation

ImportError: No module named sympy

In [2]:
# for direct kinematics

alpha1, alpha2, alpha3, alpha4, alpha5, alpha6 = symbols('alpha1:7')
a1, a2, a3, a4, a5, a6 = symbols('a1:7')
d1, d2, d3, d4, d5, d6 = symbols('d1:7')
q1, q2, q3, q4, q5, q6 = symbols('q1:7')

def set_dh_table():
    dh_subs_dict = {alpha1: pi / 2, a1: 0, d1: 0.2433, q1: q1,
                    alpha2: pi, a2: 0.28, d2: 0.03, q2: q2 + pi / 2,
                    alpha3: pi / 2, a3: 0, d3: 0.02, q3: q3 + pi / 2,
                    alpha4: pi / 2, a4: 0, d4: 0.245, q4: q4 + pi / 2,
                    alpha5: pi / 2, a5: 0, d5: 0.057, q5: q5 + pi,
                    alpha6: 0, a6: 0, d6: 0.235, q6: q6 + pi / 2}
    return dh_subs_dict

def dh(alpha, a, d, theta):
    # Formula
    return Matrix([[cos(theta), -cos(alpha)*sin(theta), sin(alpha)*sin(theta), a*cos(theta)],
                     [sin(theta), cos(alpha)*cos(theta), -sin(alpha)*cos(theta), a*sin(theta)],
                     [0, sin(alpha), cos(alpha), d],
                     [0, 0, 0, 1]])               
    
def transform_matrices():

    dictionary = set_dh_table()
    T_01 = dh(dictionary[alpha1], dictionary[a1], dictionary[d1], dictionary[q1])
    T_12 = dh(dictionary[alpha2], dictionary[a2], dictionary[d2], dictionary[q2])
    T_23 = dh(dictionary[alpha3], dictionary[a3], dictionary[d3], dictionary[q3])
    T_34 = dh(dictionary[alpha4], dictionary[a4], dictionary[d4], dictionary[q4])
    T_45 = dh(dictionary[alpha5], dictionary[a5], dictionary[d5], dictionary[q5])
    T_56 = dh(dictionary[alpha6], dictionary[a6], dictionary[d6], dictionary[q6])
    Tes = Matrix([[0, -1, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
    T = T_01 * T_12 * T_23 * T_34 * T_45 * T_56 * Tes

    return T

def FK(theta_list):

    dictionary = set_dh_table()
    T_01 = dh(dictionary[alpha1], dictionary[a1], dictionary[d1], dictionary[q1])
    T_12 = dh(dictionary[alpha2], dictionary[a2], dictionary[d2], dictionary[q2])
    T_23 = dh(dictionary[alpha3], dictionary[a3], dictionary[d3], dictionary[q3])
    T_34 = dh(dictionary[alpha4], dictionary[a4], dictionary[d4], dictionary[q4])
    T_45 = dh(dictionary[alpha5], dictionary[a5], dictionary[d5], dictionary[q5])
    T_56 = dh(dictionary[alpha6], dictionary[a6], dictionary[d6], dictionary[q6])
    Tes = Matrix([[0, -1, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
    T = T_01 * T_12 * T_23 * T_34 * T_45 * T_56 * Tes

    theta_dict = {q1:theta_list[0], q2:theta_list[1], q3:theta_list[2], q4:theta_list[3], q5:theta_list[4], q6:theta_list[5]}

    T_0G_eval = T.evalf(subs=theta_dict, chop=True, maxn=4)
    A = T_0G_eval
    A03 = np.array(A[0, 3]).astype(np.float32)
    A13 = np.array(A[1, 3]).astype(np.float32)
    A23 = np.array(A[2, 3]).astype(np.float32)
    return np.array([A03, A13, A23])


In [3]:
#data_path = set your own path to data dir
data_path = 'C:/Users/alon/Repose/robotics_kinova_lab/Lab5/data_5'
records = []
for file in glob.glob(data_path + '/*/*.pkl'):
    with open(file, 'rb') as h:
        records.append(pickle.load(h))

In [4]:
def plot_xyz(Key_points, r_path, p_path, num):

    fig = plt.figure(figsize = (55,20))
    ax1 = fig.add_subplot(2, 2, 1, projection='3d')
    ax1.scatter(Key_points[:,0],Key_points[:,1], Key_points[:,2], c='red', s=10) 
    ax1.plot(r_path[:,0], r_path[:,1], r_path[:,2], color='black')
    ax1.plot(p_path[:,0], p_path[:,1], p_path[:,2], color='blue', linestyle='dashed')
    ax1.legend(['Real path','Planned path', 'Key points'])
    ax1.set_xlabel('X[m]')
    ax1.set_ylabel('Y[m]')
    ax1.set_zlabel('Z[m]')
    ax1.set_title(f"XYZ Gripper path, {num}")

    # plt.savefig(f"C:/Users/alon/Repose/robotics_kinova_lab/Lab4/figs/{num}.png")

    plt.show()

In [ ]:
records[0][0] # robot angles 
records[0][1] # gripper pose 
records[0][2] # euler angles

In [18]:
len(records[9])


3